In [1]:
import datetime
import pandas
import psycopg2
import os
import matplotlib.pyplot as plt

conn = psycopg2.connect(host = '127.0.0.1' , user = 'postgres' , dbname = 'postgres' , password = 'mon3636a')
cur = conn.cursor()
pandas.set_option('display.max_rows',5000)

# regulation
### 連續四天20日集中度上升 

In [48]:
insert_sql = "insert into report.cct20_history \
              (date,no,name,price,cct5,cct20,fi_obs,it_obs,qty10,exponent,topic,tangled_num) \
              values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
insert_c_sql = "insert into report.cct20_history_c \
              (date,no,name,price,cct5,cct20,fi_obs,it_obs,qty10,exponent,topic,tangled_num) \
              values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"

In [2]:
sma = pandas.read_sql("select date,no,close,sma5,sma10,sma20,sma60 from ods.analyze_avg where date_no = (select max(date_no) from work_date)" , con = conn)
sma['糾結數量'] = 0
i = 0
for i in range(len(sma)):
    if (sma['sma5'][i] <= sma['sma10'][i]*1.03 and sma['sma5'][i] >= sma['sma10'][i]*0.97) and \
    (sma['sma5'][i] <= sma['sma20'][i]*1.03 and sma['sma5'][i] >= sma['sma20'][i]*0.97) and \
    (sma['sma5'][i] <= sma['sma60'][i]*1.03 and sma['sma5'][i] >= sma['sma60'][i]*0.97):
        sma['糾結數量'].iloc[i] = 4
    elif (sma['sma5'][i] <= sma['sma10'][i]*1.03 and sma['sma5'][i] >= sma['sma10'][i]*0.97) and \
    (sma['sma5'][i] <= sma['sma20'][i]*1.03 and sma['sma5'][i] >= sma['sma20'][i]*0.97) :
        sma['糾結數量'].iloc[i] = 3
sma = sma[['no','糾結數量']]
sma.columns = ['股票代號','糾結數量']

/home/buneo/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [62]:
a = pandas.read_sql("select main.* , fi.sum20 from (select *  from ods.cct20 c \
                  where c.date_no >= (select max(date_no) from work_date )-4 ) main join ods.analyze_fi_obs fi on main.date = fi.date and main.no = fi.no\
                  order by main.no , main.date_no\
                " , con = conn)

a['bigger'] = ''
query_D = pandas.read_sql("select max(date) from work_date" , con = conn).iloc[0,0]

try:
    os.mkdir('/home/buneo/Stock/CCT20/{:0>4}{:0>2}{:0>2}'.format(query_D.year , query_D.month , query_D.day))
    pass
except FileExistsError:
    pass 
for i in range(1,len(a)):
    if a['no'].iloc[i] == a['no'].iloc[i-1]:
        if a['cct20'].iloc[i] > a['cct20'].iloc[i-1]:
            a['bigger'].iloc[i] = 'Y'
b = a[a['bigger'] == 'Y'][['no','bigger']].groupby('no').count()
qqq = b[b['bigger'] == 4].index.tolist()
c =a[a['date'] == query_D]
mask = c['no'].isin(qqq)
clas = pandas.read_sql("select no , concat(classify,'/') topic from classify where type in ('topic','semi')" , con = conn).groupby('no').sum('topic')
p = pandas.read_sql("select date , no , close , name from stock_daily where date = '{0}' union select date , no , close , name from otc_daily where date = '{0}' ".format(query_D) , con = conn)
it20 = pandas.read_sql("select no,date,sum20 as IT20 from ods.analyze_it_obs where date = '{}'".format(query_D) , con = conn)
c = pandas.merge(c[['date','no','cct20','sum20']],clas,on='no',how = 'left')
c = pandas.merge(c,p , on ='no').iloc[:,[0,1,7,6,2,3,4]]
c = pandas.merge(c,it20 , on = 'no').iloc[:,[0,1,2,3,4,5,8,6]]

exp = pandas.read_sql("select no AS 股票代號 , classify AS 類股 from classify where type in ('otc','stock')" , con = conn)

c.columns = ['日期','股票代號','股票名稱','收盤價','20日集中度','20日外資買賣超','20日投信買賣超','題材']
c = pandas.merge(c,sma,on = '股票代號',how = 'left')
c = pandas.merge(c,exp,on = '股票代號' , how ='left')
c = c.iloc[:,[0,1,2,3,4,5,6,9,7,8]]
c5 = pandas.read_sql("select no , cct5 from ods.cct5 where date = '{}'".format(query_D) , con = conn)

#c.append(pandas.DataFrame([['糾結邏輯:'],['3 : sma5 * 0.97 <= sma10 & sma20 <= sma5 *1.03'],['4 : sma5 * 0.97 <= sma10 & sma20 & sma60 <= sma5 *1.03']]))

qty10 = pandas.read_sql("select  no , cast(avg(quantity) as int) AS qty10 from stock_daily stock where stock.date in (select date from work_date where date_no >= (select max(date_no) from work_date ) -9 )  group by no union \
                        select  no , cast(avg(quantity) as int )AS qty10 from otc_daily stock where stock.date in (select date from work_date where date_no >= (select max(date_no) from work_date ) -9 )  group by no " , con = conn)
cc10 = pandas.merge(c[c['股票代號'].isin(qqq)],qty10,left_on = '股票代號',right_on = 'no')


ms1 = cc10['股票代號'].isin(qqq)
ms2 = cc10['20日集中度'] <= 0.5
ms3 = cc10['20日集中度'] >= -0.05
ms4 = cc10['qty10'] >= 500
report_cct10 = cc10[ms1 & ms2 & ms3 & ms4]

report_cct10 = pandas.merge(left = report_cct10 , right = c5 , left_on = '股票代號',right_on = 'no').iloc[:,[0,1,2,3,13,4,5,6,11,7,8,9]]
report_cct10.columns = ['日期','股票代號','股票名稱','收盤價','5日集中度','20日集中度','20日外資買賣超','20日投信買賣超','10日平均成交量','類股','題材','糾結數量']

cur.execute("delete from report.cct20_history where date = '{}'".format(str(report_cct10['日期'].iloc[0])))
cur.execute("commit")

for _ , data in report_cct10.iterrows():
    cur.execute(insert_sql,data)
cur.execute("commit")

report_cct10.sort_values('20日集中度').to_csv('/home/buneo/Stock/CCT20/{0:0>4}{1:0>2}{2:0>2}/20日集中度_{1:0>2}{2:0>2}.csv'.format(query_D.year, query_D.month,query_D.day),index = False)

/home/buneo/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [60]:
qty5 = pandas.read_sql("select  no , avg(quantity) qty from stock_daily stock where stock.date in (select date from work_date where date_no >= (select max(date_no) from work_date ) -4 )  group by no union \
                        select  no , avg(quantity) qty from otc_daily stock where stock.date in (select date from work_date where date_no >= (select max(date_no) from work_date ) -4 )  group by no " , con = conn)
cc5 = pandas.merge(c[c['股票代號'].isin(qqq)],qty5,left_on = '股票代號',right_on = 'no')

mask_1 = cc5['qty']>=1500
mask_2 = cc5['收盤價'] >=5
mask_3 = cc5['收盤價'] <=150
mask_4 = cc5['20日集中度'] >= -0.1
report_cct5 = cc5[mask_1 & mask_2 & mask_3 & mask_4].iloc[:,[0,1,2,3,11,4,5,6,7,8,9]]
report_cct5 = pandas.merge(left = report_cct5 , right = c5 , left_on = '股票代號' , right_on = 'no').iloc[:,[0,1,2,3,12,5,6,7,4,8,9,10]]

cur.execute("delete from report.cct20_history_c where date = '{}'".format(str(report_cct5['日期'].iloc[0])))
cur.execute("commit")

for _ , data in report_cct5.iterrows():
    cur.execute(insert_c_sql,data)
cur.execute("commit")
report_cct5.columns = ['日期','股票代號','股票名稱','收盤價','5日集中度','20日集中度','20日外資買賣超','20日投信買賣超','10日平均成交量','類股','題材','糾結數量']
report_cct5.sort_values('20日集中度').to_csv('/home/buneo/Stock/CCT20/{0:0>4}{1:0>2}{2:0>2}/20日集中度_{1:0>2}{2:0>2}_鄭大版.csv'.format(query_D.year, query_D.month,query_D.day),index = False)